In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-17"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12054,2024-04-17,Macedônia Do Norte Prva Liga,13:00,FMP Akademija,Kozuv,1.92,1.80,166.5,1.86,1.80,0.0,0.00,0.00,p6LnVn6i,0.520833,0.555556,0.537634,0.555556,0.076389,599.888,572.163550,0.953784,0.6,1.341641,2.236068,181.24,7.272,6.641692,0.913324,-64.0,308.694,345.552186,1.119400,1.2,1.643168,1.369306,102.83,4.106,4.789570,1.166481,-14.0,92,91,1.97,1.13,335.636,309.632,0.045620,0.023184,NaN,-2.37,-0.474,-1.940928,0.000000,0.0,0.000000,-3.88,-0.776,-1.030928,0.000000,0.0,0.000000
12055,2024-04-17,República Tcheca Nbl,13:00,Usti n. Labem,Pardubice,1.22,3.98,165.5,1.81,1.89,0.0,0.00,0.00,fPGv6gFl,0.819672,0.251256,0.552486,0.529101,0.070928,196.700,137.981876,0.701484,1.8,1.643168,0.912871,103.32,2.428,2.054889,0.846330,33.0,279.324,169.778065,0.607818,0.0,0.000000,NaN,247.04,3.996,2.343476,0.586455,-84.0,84,64,1.23,3.86,158.550,317.964,0.750621,0.030578,NaN,3.77,0.754,0.291777,0.780205,1.0,0.219795,1.39,0.278,10.719424,0.343159,0.7,0.356841
12056,2024-04-17,República Tcheca Nbl,13:00,Decin,Brno,1.41,2.76,162.5,1.81,1.89,0.0,0.00,0.00,zqxEKhUs,0.709220,0.362319,0.552486,0.529101,0.071539,191.574,148.557936,0.775460,2.4,1.341641,0.559017,95.91,2.392,1.565861,0.654624,-5.0,238.122,132.146164,0.554952,0.6,1.341641,2.236068,181.76,3.212,1.900768,0.591771,-32.0,69,64,1.39,2.84,190.674,308.062,0.457839,0.030578,NaN,4.71,0.942,0.435244,0.614158,0.7,0.085842,-3.57,-0.714,-2.464986,0.316814,0.3,-0.016814
12057,2024-04-17,Nova Zelândia Nbl,04:30,Nelson Giants,Manawatu Jets,1.18,4.42,174.5,1.86,1.84,0.0,0.00,0.00,U1DLHTpU,0.847458,0.226244,0.537634,0.543478,0.073702,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,289.150,52.814294,0.182654,0.6,1.341641,2.236068,366.86,3.354,0.760316,0.226689,-22.0,88,83,2.97,4.42,0.000,0.000,0.818224,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12058,2024-04-17,México Cibacopa,00:15,Ostioneros de Guaymas,Venados de Mazatlan,1.71,2.03,178.5,1.86,1.84,-2.5,1.94,2.50,z1yy1ijg,0.584795,0.492611,0.537634,0.543478,0.077406,218.930,83.025777,0.379234,1.8,1.643168,0.912871,255.64,2.462,0.780077,0.316847,-22.0,232.168,94.551705,0.407256,1.8,1.643168,0.912871,146.94,2.774,1.233017,0.444490,5.0,83,93,3.08,1.58,227.224,277.444,0.121002,0.007644,0.178369,-0.27,-0.054,-13.148148,0.512521,0.7,0.187479,1.00,0.200,5.150000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,2024-04-17,Finlândia Korisliiga Feminina,12:30,Torpan Pojat Helsinki F,PeKa F,1.06,8.00,159.5,1.85,1.85,0.0,0.00,0.00,dSBLog7e,0.943396,0.125000,0.540541,0.540541,0.068396,96.530,6.941045,0.071906,2.4,1.341641,0.559017,105.09,1.058,0.044385,0.041951,73.0,110.390,14.915740,0.135119,2.4,1.341641,0.559017,126.90,1.278,0.090940,0.071158,60.0,93,94,1.13,1.35,715.856,124.958,1.083294,0.000000,NaN,28.15,5.630,0.010657,0.000000,0.0,0.000000,-0.90,-0.180,-38.888889,0.720157,0.7,-0.020157
12140,2024-04-17,Filipinas Copa Das Filipinas,08:30,Magnolia Hotshots,Blackwater Bossing,1.18,4.69,183.5,1.87,1.86,0.0,0.00,0.00,E3EwOGIi,0.847458,0.213220,0.534759,0.537634,0.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12089,21:00,Uruguai Liga Uruguaia,Hebraica y Macabi,Nacional,1.59,Back Home
12103,07:00,Coreia Do Sul Kbl,Wonju DB,KCC Egis,1.58,Back Home
12112,14:30,Alemanha Pro A,Vechta 2,Koblenz,1.55,Back Home
12120,07:05,Japão B.League,Kyoto,Shimane,2.65,Back Home
